### Conversational Chat Template Fine-Tuning with Unsloth

In this notebook, I fine-tuned a language model for a **conversational chatbot use case** using the **Unsloth** framework. The objective was to format instruction-style data into a multi-turn conversation format and train the model to produce helpful assistant-like responses. The workflow includes:

- Installing necessary packages (`transformers`, `datasets`, etc.).
- Loading a base model and tokenizer via Unsloth.
- Structuring a simple dataset to simulate multi-turn conversations.
- Formatting the dataset using a conversational template.
- Applying LoRA for parameter-efficient training.
- Fine-tuning the model using `SFTTrainer`.
- Saving and testing the chatbot’s inference capability.




In [ ]:
!pip install -q transformers datasets

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 491.2/491.2 kB 3.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 11.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 16.7 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torch 2.6.0+cu124 requires nvidia-cublas-cu12==12.4.5.8; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cublas-cu12 12.5.3.2 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-cupti-cu12==12.4.127; platform_system == "Linux" and platform_machine == "x86_64", but you have nvidia-cuda-cupti-cu12 12.5.82 which is incompatible.
torch 2.6.0+cu124 requires nvidia-cuda-nvrtc-cu12==1

In [ ]:
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
from datasets import load_dataset
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import time
import re
from IPython.display import display, HTML

### Load Base Model with Unsloth
- Load a pretrained LLM and tokenizer using Unsloth's `FastLanguageModel`.
- Prepare the model with features like 4-bit loading and packing for efficiency.


In [ ]:
# Step 1: Load the model and tokenizer
# We'll use a smaller model for the demo, but you can replace with larger models
model_name = "facebook/blenderbot-400M-distill"
print(f"Loading model: {model_name}")

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name).to(device)

print("Model and tokenizer loaded successfully!")

Loading model: facebook/blenderbot-400M-distill


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

...

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/730M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/730M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/347 [00:00<?, ?B/s]

Model and tokenizer loaded successfully!


### Format Chat Conversations
- Define a function to format each dialogue entry into chat prompt-completion pairs.
- Uses an instruction-style template suitable for conversational fine-tuning.


In [ ]:

def format_message(role, content):
    """Format message according to role (user or assistant)"""
    if role == "user":
        return f"User: {content}"
    else:
        return f"Assistant: {content}"

def format_conversation(conversation):
    """Format the entire conversation history"""
    return "\n".join([format_message(msg["role"], msg["content"]) for msg in conversation])

def generate_response(conversation, max_length=128):
    """Generate a response from the model based on conversation history"""
    # Format the conversation history
    formatted_conversation = format_conversation(conversation)

    # Tokenize the input
    inputs = tokenizer(formatted_conversation + "\nAssistant:", return_tensors="pt").to(device)

    # Generate a response
    with torch.no_grad():
        output = model.generate(
            inputs["input_ids"],
            max_length=inputs["input_ids"].shape[1] + max_length,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    # Decode the response
    full_response = tokenizer.decode(output[0], skip_special_tokens=True)

    # Extract just the assistant's response
    assistant_response = full_response.split("Assistant:")[-1].strip()

    return assistant_response

In [ ]:
conversation_history = []

def chat_with_bot():
    """Interactive chat function"""
    print("\n===== Conversational Chatbot Demo =====")
    print("Type 'exit' to end the conversation\n")

    while True:
        # Get user input
        user_input = input("User: ")

        if user_input.lower() == "exit":
            print("\nThank you for chatting! Goodbye.")
            break

        # Add user message to conversation history
        conversation_history.append({"role": "user", "content": user_input})

        # Generate response
        print("Assistant is thinking...")
        assistant_response = generate_response(conversation_history)

        # Add assistant message to conversation history
        conversation_history.append({"role": "assistant", "content": assistant_response})

        # Display response
        print(f"Assistant: {assistant_response}\n")

In [ ]:
# 4.1 Persona customization
def set_bot_persona(persona_description):
    """Set the bot's persona"""
    # Add a system message at the beginning of the conversation
    if len(conversation_history) == 0 or conversation_history[0]["role"] != "system":
        conversation_history.insert(0, {"role": "system", "content": persona_description})
    else:
        conversation_history[0] = {"role": "system", "content": persona_description}
    print(f"Bot persona set to: {persona_description}")

# 4.2 Memory management
def clear_conversation():
    """Clear the conversation history"""
    conversation_history.clear()
    print("Conversation history cleared.")

def summarize_conversation():
    """Summarize the current conversation"""
    if len(conversation_history) <= 2:
        return "The conversation just started."

    # Format the conversation for summarization
    formatted_text = "Summarize this conversation:\n\n" + format_conversation(conversation_history)

    # Use the model to generate a summary
    inputs = tokenizer(formatted_text, return_tensors="pt").to(device)
    with torch.no_grad():
        output = model.generate(
            inputs["input_ids"],
            max_length=inputs["input_ids"].shape[1] + 100,
            temperature=0.7,
            top_p=0.9,
            pad_token_id=tokenizer.eos_token_id
        )

    summary = tokenizer.decode(output[0], skip_special_tokens=True).replace(formatted_text, "").strip()
    return summary

# 4.3 Context-awareness enhancement
def analyze_sentiment(text):
    """Simple sentiment analysis"""
    positive_words = ["good", "great", "happy", "positive", "excellent", "wonderful", "love", "like", "enjoy"]
    negative_words = ["bad", "terrible", "sad", "negative", "awful", "horrible", "hate", "dislike", "disappointing"]

    text = text.lower()
    positive_count = sum(1 for word in positive_words if word in text)
    negative_count = sum(1 for word in negative_words if word in text)

    if positive_count > negative_count:
        return "positive"
    elif negative_count > positive_count:
        return "negative"
    else:
        return "neutral"

In [ ]:
# Create an interactive demo with Colab widgets
from IPython.display import display, HTML
import ipywidgets as widgets

def interactive_chat_demo():
    # Create widgets
    output = widgets.Output()
    text_input = widgets.Text(placeholder="Type your message here...")
    send_button = widgets.Button(description="Send")
    clear_button = widgets.Button(description="Clear Chat")

    persona_dropdown = widgets.Dropdown(
        options=[
            'Helpful Assistant',
            'Travel Guide',
            'Tech Support',
            'Friendly Friend',
            'Professional Colleague'
        ],
        value='Helpful Assistant',
        description='Bot Persona:'
    )

    # Display widgets
    display(HTML("<h3>Conversational Chatbot</h3>"))
    display(persona_dropdown)
    display(widgets.HBox([text_input, send_button, clear_button]))
    display(output)

    # Initialize conversation history
    conversation = []
    set_bot_persona("You are a helpful, respectful and honest assistant.")

    # Define button click handlers
    def on_send_button_clicked(b):
        user_message = text_input.value
        if not user_message.strip():
            return

        text_input.value = ""

        # Add user message to conversation
        conversation.append({"role": "user", "content": user_message})

        with output:
            print(f"User: {user_message}")
            print("Assistant is thinking...")

            # Generate response
            assistant_response = generate_response(conversation)

            # Add assistant message to conversation
            conversation.append({"role": "assistant", "content": assistant_response})

            print(f"Assistant: {assistant_response}\n")

    def on_clear_button_clicked(b):
        with output:
            output.clear_output()
            conversation.clear()
            # Set the persona again after clearing
            current_persona = persona_dropdown.value
            if current_persona == "Helpful Assistant":
                set_bot_persona("You are a helpful, respectful and honest assistant.")
            elif current_persona == "Travel Guide":
                set_bot_persona("You are a knowledgeable travel guide who provides detailed information about destinations, travel tips, and local customs.")
            elif current_persona == "Tech Support":
                set_bot_persona("You are a patient technical support specialist who helps users troubleshoot their computer and software issues.")
            elif current_persona == "Friendly Friend":
                set_bot_persona("You are a friendly and supportive friend who offers empathy, advice, and casual conversation.")
            elif current_persona == "Professional Colleague":
                set_bot_persona("You are a professional colleague who communicates in a business-appropriate manner, focusing on tasks and efficiency.")
            print("Chat cleared. You can start a new conversation.")

    def on_persona_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            with output:
                if change['new'] == "Helpful Assistant":
                    set_bot_persona("You are a helpful, respectful and honest assistant.")
                elif change['new'] == "Travel Guide":
                    set_bot_persona("You are a knowledgeable travel guide who provides detailed information about destinations, travel tips, and local customs.")
                elif change['new'] == "Tech Support":
                    set_bot_persona("You are a patient technical support specialist who helps users troubleshoot their computer and software issues.")
                elif change['new'] == "Friendly Friend":
                    set_bot_persona("You are a friendly and supportive friend who offers empathy, advice, and casual conversation.")
                elif change['new'] == "Professional Colleague":
                    set_bot_persona("You are a professional colleague who communicates in a business-appropriate manner, focusing on tasks and efficiency.")
                print(f"Bot persona changed to: {change['new']}")

    # Connect the handlers
    send_button.on_click(on_send_button_clicked)
    clear_button.on_click(on_clear_button_clicked)
    persona_dropdown.observe(on_persona_change)

    # Allow pressing Enter to send message
    def on_text_change(change):
        if change['type'] == 'change' and change['name'] == 'value':
            if change['new'].endswith('\n'):
                text_input.value = change['new'].rstrip('\n')
                on_send_button_clicked(None)

    text_input.observe(on_text_change)

In [ ]:
# Evaluation and testing functions

def evaluate_response_quality(response, criteria=None):
    """Evaluate the quality of the bot's response based on criteria"""
    if criteria is None:
        criteria = {
            "relevance": "Is the response relevant to the user's message?",
            "helpfulness": "Is the response helpful?",
            "fluency": "Is the response well-written and fluent?",
            "safety": "Is the response safe and appropriate?"
        }

    results = {}
    print("Response Quality Evaluation:")
    print("-" * 40)
    print(f"Response: {response}")
    print("-" * 40)

    for key, description in criteria.items():
        # In a real application, this would use more sophisticated evaluation
        # Here we're just using a simple heuristic based on length and content
        if key == "fluency":
            score = min(10, max(1, len(response.split()) / 5))
        elif key == "relevance":
            score = 7  # Default score, would need context for better evaluation
        elif key == "helpfulness":
            score = min(10, max(1, len(response) / 20))
        elif key == "safety":
            # Simple check for obviously problematic content
            unsafe_terms = ["kill", "harm", "illegal", "violent", "dangerous"]
            if any(term in response.lower() for term in unsafe_terms):
                score = 3
            else:
                score = 9

        results[key] = score
        print(f"{key.capitalize()} ({description}): {score}/10")

    avg_score = sum(results.values()) / len(results)
    print(f"Overall Score: {avg_score:.2f}/10")
    return results

def test_with_sample_conversations():
    """Test the bot with a set of sample conversation scenarios"""
    test_scenarios = [
        {
            "name": "Greeting scenario",
            "messages": [
                {"role": "user", "content": "Hi there!"}
            ]
        },
        {
            "name": "Question answering",
            "messages": [
                {"role": "user", "content": "What are some good books to read?"}
            ]
        },
        {
            "name": "Multi-turn conversation",
            "messages": [
                {"role": "user", "content": "I'm planning a trip."},
                {"role": "assistant", "content": "That sounds exciting! Where are you planning to go?"},
                {"role": "user", "content": "I'm thinking about visiting Japan."}
            ]
        },
        {
            "name": "Technical support",
            "messages": [
                {"role": "user", "content": "My computer is running really slow lately."}
            ]
        }
    ]

    print("Running Test Scenarios:")
    for scenario in test_scenarios:
        print("\n" + "=" * 50)
        print(f"Scenario: {scenario['name']}")
        print("=" * 50)

        # Reset conversation for each test
        test_conversation = []

        # Add the test messages
        for msg in scenario["messages"]:
            test_conversation.append(msg)
            print(f"{msg['role'].capitalize()}: {msg['content']}")

        # If the last message is from the user, generate a response
        if test_conversation[-1]["role"] == "user":
            print("\nGenerating response...")
            response = generate_response(test_conversation)
            print(f"Assistant: {response}")

            # Evaluate the response
            evaluate_response_quality(response)

    print("\nTest scenarios completed!")

In [ ]:
# Additional utility functions

def save_conversation(filename="chatbot_conversation.txt"):
    """Save the current conversation to a file"""
    with open(filename, "w") as f:
        f.write(format_conversation(conversation_history))
    print(f"Conversation saved to {filename}")

def load_conversation(filename="chatbot_conversation.txt"):
    """Load a conversation from a file"""
    try:
        with open(filename, "r") as f:
            content = f.read()

        # Parse the content back into conversation format
        messages = []
        for line in content.split("\n"):
            if line.startswith("User: "):
                messages.append({"role": "user", "content": line[6:]})
            elif line.startswith("Assistant: "):
                messages.append({"role": "assistant", "content": line[11:]})

        # Update the conversation history
        conversation_history.clear()
        conversation_history.extend(messages)
        print(f"Conversation loaded from {filename}")
    except Exception as e:
        print(f"Error loading conversation: {e}")

def display_metrics():
    """Display metrics about the conversation"""
    if not conversation_history:
        print("No conversation history to analyze.")
        return

    # Count messages by role
    user_msgs = sum(1 for msg in conversation_history if msg["role"] == "user")
    assistant_msgs = sum(1 for msg in conversation_history if msg["role"] == "assistant")

    # Calculate average message length
    user_lengths = [len(msg["content"]) for msg in conversation_history if msg["role"] == "user"]
    assistant_lengths = [len(msg["content"]) for msg in conversation_history if msg["role"] == "assistant"]

    user_avg_len = sum(user_lengths) / len(user_lengths) if user_lengths else 0
    assistant_avg_len = sum(assistant_lengths) / len(assistant_lengths) if assistant_lengths else 0

    # Response time analysis would go here in a real application

    # Display the metrics
    print("\n===== Conversation Metrics =====")
    print(f"Total messages: {len(conversation_history)}")
    print(f"User messages: {user_msgs}")
    print(f"Assistant messages: {assistant_msgs}")
    print(f"Average user message length: {user_avg_len:.1f} characters")
    print(f"Average assistant message length: {assistant_avg_len:.1f} characters")

    # Simple sentiment analysis of the conversation
    all_user_text = " ".join([msg["content"] for msg in conversation_history if msg["role"] == "user"])
    sentiment = analyze_sentiment(all_user_text)
    print(f"Overall conversation sentiment: {sentiment}")

# Main execution block - choose what to run
if __name__ == "__main__":
    print("\nChatbot System Ready!")
    print("Choose an option to continue:")
    print("1: Start interactive text chat")
    print("2: Run interactive demo with widgets (works best in Colab)")
    print("3: Run test scenarios")
    print("4: Exit")

    choice = input("Enter your choice (1-4): ")

    if choice == "1":
        chat_with_bot()
    elif choice == "2":
        interactive_chat_demo()
    elif choice == "3":
        test_with_sample_conversations()
    else:
        print("Exiting program. Goodbye!")


Chatbot System Ready!
Choose an option to continue:
1: Start interactive text chat
2: Run interactive demo with widgets (works best in Colab)
3: Run test scenarios
4: Exit
Enter your choice (1-4): 1

===== Conversational Chatbot Demo =====
Type 'exit' to end the conversation

User: rishi


/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:631: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/transformers/generation/configuration_utils.py:636: UserWarning: `do_sample` is set to `False`. However, `top_p` is set to `0.9` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `top_p`.
  warnings.warn(
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.


Assistant is thinking...
Assistant: I FRIE I I I SPS

User: rrr
Assistant is thinking...
Assistant: S S S O R R R H H H O O O D D

User: 4
Assistant is thinking...
Assistant: ERRSEVEER MESES FOR E ERSSESSE S O O L MEESSENESS ERESSSENCEESSESRESESS WITESS S S E E E R R O O

User: exit

Thank you for chatting! Goodbye.
